In [54]:
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import StandardScaler

In [55]:
airline_df = pd.read_csv("cleaned_airline_2018.csv")

abe_weather_df = pd.read_csv("abe_cleaned_weather_2018.csv")
atl_weather_df = pd.read_csv("atl_cleaned_weather_2018.csv")

In [56]:
airline_df.head()

,Date,Origin,Destination,Delay,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,Endeavor Air,Envoy Air,...,Hawaiian Airlines,JetBlue Airways,Mesa Airline,PSA Airlines,Republic Airways,SkyWest Airlines,Southwest Airlines,Spirit Airlines,United Airlines,Virgin America
0,2018-01-01,EWR,DEN,-18.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2018-01-01,LAS,SFO,-16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2018-01-01,SNA,DEN,-8.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2018-01-01,RSW,ORD,-8.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2018-01-01,ORD,ALB,-6.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [58]:
abe_weather_df.head()

,Date,Precipitation,Rain,Snowfall,Windspeed,Windgusts,Evapotranspiration
0,2018-01-01,0.0,0.0,0.00,17.9,43.2,0.81
1,2018-01-02,0.0,0.0,0.00,16.0,38.9,0.89
2,2018-01-03,0.0,0.0,0.00,9.3,22.7,0.85
3,2018-01-04,10.2,0.0,7.28,29.8,67.0,0.74
4,2018-01-05,0.0,0.0,0.00,30.9,67.3,0.92


In [59]:
airline_df['Origin'].value_counts()

ATL    386586
ORD    324908
DFW    273243
DEN    233741
CLT    226795
        ...  
AKN        63
CYS        57
IFP        45
ART        24
YNG         2
Name: Origin, Length: 358, dtype: int64

In [61]:
airline_df[airline_df['Origin'] == 'ABE']['Destination'].value_counts()

ATL    960
DTW    912
CLT    911
SFB    441
ORD    345
PIE    157
PGD    144
MYR     84
FLL     72
PHL     40
Name: Destination, dtype: int64

In [62]:
abe_atl_airline_df = airline_df[(airline_df['Origin'] == 'ABE') & (airline_df['Destination'] == 'ATL')]
abe_atl_airline_df.head()

,Date,Origin,Destination,Delay,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,Endeavor Air,Envoy Air,...,Hawaiian Airlines,JetBlue Airways,Mesa Airline,PSA Airlines,Republic Airways,SkyWest Airlines,Southwest Airlines,Spirit Airlines,United Airlines,Virgin America
3936,2018-01-01,ABE,ATL,21.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19725,2018-01-02,ABE,ATL,-22.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19733,2018-01-02,ABE,ATL,-15.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40933,2018-01-03,ABE,ATL,5.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54149,2018-01-03,ABE,ATL,19.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
origin_weather_cols = {
    'Precipitation': 'Origin Precipitation', 
    'Rain': 'Origin Rain', 
    'Snowfall': 'Origin Snowfall',
    'Windspeed': 'Origin Windspeed', 
    'Windgusts': 'Origin Windgusts',
    'Evapotranspiration': 'Origin Evapotranspiration'
}

dest_weather_cols = {
    'Precipitation': 'Dest Precipitation', 
    'Rain': 'Dest Rain', 
    'Snowfall': 'Dest Snowfall',
    'Windspeed': 'Dest Windspeed', 
    'Windgusts': 'Dest Windgusts',
    'Evapotranspiration': 'Dest Evapotranspiration'
}

abe_atl_airline_df = abe_atl_airline_df.join(abe_weather_df.set_index('Date'), on='Date')
abe_atl_airline_df.rename(columns = origin_weather_cols, inplace = True)

abe_atl_airline_df = abe_atl_airline_df.join(abe_weather_df.set_index('Date'), on='Date')
abe_atl_airline_df.rename(columns = dest_weather_cols, inplace = True)

abe_atl_airline_df.head()

,Date,Origin,Destination,Delay,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,Endeavor Air,Envoy Air,...,Origin Snowfall,Origin Windspeed,Origin Windgusts,Origin Evapotranspiration,Dest Precipitation,Dest Rain,Dest Snowfall,Dest Windspeed,Dest Windgusts,Dest Evapotranspiration
3936,2018-01-01,ABE,ATL,21.0,0,0,0,0,0,0,...,0.0,17.9,43.2,0.81,0.0,0.0,0.0,17.9,43.2,0.81
19725,2018-01-02,ABE,ATL,-22.0,0,0,0,0,0,0,...,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
19733,2018-01-02,ABE,ATL,-15.0,0,0,0,0,0,0,...,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
40933,2018-01-03,ABE,ATL,5.0,0,0,0,0,0,0,...,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85
54149,2018-01-03,ABE,ATL,19.0,0,0,0,1,0,0,...,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85


In [64]:
abe_atl_airline_df.describe()

,Delay,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,Endeavor Air,Envoy Air,ExpressJet,Frontier Airlines,Hawaiian Airlines,...,Origin Snowfall,Origin Windspeed,Origin Windgusts,Origin Evapotranspiration,Dest Precipitation,Dest Rain,Dest Snowfall,Dest Windspeed,Dest Windgusts,Dest Evapotranspiration
count,960.000000,960.0,960.0,960.0,960.000000,960.000000,960.0,960.000000,960.0,960.0,...,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000,960.000000
mean,-3.302083,0.0,0.0,0.0,0.376042,0.365625,0.0,0.258333,0.0,0.0,...,0.226042,15.300625,37.238542,2.590219,4.338021,4.021042,0.226042,15.300625,37.238542,2.590219
std,14.845692,0.0,0.0,0.0,0.484643,0.481856,0.0,0.437946,0.0,0.0,...,1.465332,5.359266,12.854702,1.743023,8.085590,7.841272,1.465332,5.359266,12.854702,1.743023
min,-37.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,6.800000,13.000000,0.230000,0.000000,0.000000,0.000000,6.800000,13.000000,0.230000
25%,-12.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,11.100000,27.400000,1.200000,0.000000,0.000000,0.000000,11.100000,27.400000,1.200000
50%,-6.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,14.350000,35.300000,2.110000,0.400000,0.200000,0.000000,14.350000,35.300000,2.110000
75%,3.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,0.0,0.0,...,0.000000,18.400000,43.900000,3.830000,4.600000,4.000000,0.000000,18.400000,43.900000,3.830000
max,106.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,0.0,0.0,...,14.560000,37.800000,88.600000,7.190000,55.700000,55.700000,14.560000,37.800000,88.600000,7.190000


In [65]:
abe_atl_airline_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 960 entries, 3936 to 7069364
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       960 non-null    object 
 1   Origin                     960 non-null    object 
 2   Destination                960 non-null    object 
 3   Delay                      960 non-null    float64
 4   Alaska Airlines            960 non-null    int64  
 5   Allegiant Air              960 non-null    int64  
 6   American Airlines          960 non-null    int64  
 7   Delta Airlines             960 non-null    int64  
 8   Endeavor Air               960 non-null    int64  
 9   Envoy Air                  960 non-null    int64  
 10  ExpressJet                 960 non-null    int64  
 11  Frontier Airlines          960 non-null    int64  
 12  Hawaiian Airlines          960 non-null    int64  
 13  JetBlue Airways            960 non-null    

In [67]:
without_airline_cols = [ 
    'Delay', 'Origin Precipitation', 'Origin Rain', 'Origin Snowfall', 
    'Origin Windspeed', 'Origin Windgusts', 'Origin Evapotranspiration', 
    'Dest Precipitation', 'Dest Rain', 'Dest Snowfall', 'Dest Windspeed', 
    'Dest Windgusts', 'Dest Evapotranspiration'
]

features_df = abe_atl_airline_df[without_airline_cols]
features_df.head()

,Delay,Origin Precipitation,Origin Rain,Origin Snowfall,Origin Windspeed,Origin Windgusts,Origin Evapotranspiration,Dest Precipitation,Dest Rain,Dest Snowfall,Dest Windspeed,Dest Windgusts,Dest Evapotranspiration
3936,21.0,0.0,0.0,0.0,17.9,43.2,0.81,0.0,0.0,0.0,17.9,43.2,0.81
19725,-22.0,0.0,0.0,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
19733,-15.0,0.0,0.0,0.0,16.0,38.9,0.89,0.0,0.0,0.0,16.0,38.9,0.89
40933,5.0,0.0,0.0,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85
54149,19.0,0.0,0.0,0.0,9.3,22.7,0.85,0.0,0.0,0.0,9.3,22.7,0.85


In [68]:
scaler = StandardScaler()
scaled_features_np = scaler.fit_transform(features_df)
scaled_features_df = pd.DataFrame(scaled_features_np , columns=without_airline_cols)

scaled_features_df.head()

,Delay,Origin Precipitation,Origin Rain,Origin Snowfall,Origin Windspeed,Origin Windgusts,Origin Evapotranspiration,Dest Precipitation,Dest Rain,Dest Snowfall,Dest Windspeed,Dest Windgusts,Dest Evapotranspiration
0,1.637832,-0.536792,-0.513072,-0.15434,0.485277,0.463999,-1.021872,-0.536792,-0.513072,-0.15434,0.485277,0.463999,-1.021872
1,-1.260141,-0.536792,-0.513072,-0.15434,0.130566,0.129316,-0.975951,-0.536792,-0.513072,-0.15434,0.130566,0.129316,-0.975951
2,-0.788378,-0.536792,-0.513072,-0.15434,0.130566,0.129316,-0.975951,-0.536792,-0.513072,-0.15434,0.130566,0.129316,-0.975951
3,0.559517,-0.536792,-0.513072,-0.15434,-1.120257,-1.131580,-0.998912,-0.536792,-0.513072,-0.15434,-1.120257,-1.131580,-0.998912
4,1.503043,-0.536792,-0.513072,-0.15434,-1.120257,-1.131580,-0.998912,-0.536792,-0.513072,-0.15434,-1.120257,-1.131580,-0.998912


In [69]:
fig = px.imshow(
    scaled_features_df.corr(),
    aspect="auto"
)
fig.show()

In [72]:
fig = px.histogram(scaled_features_df, x='Delay')
fig.show()

In [73]:
scaled_features_df.to_csv('abe_atl_eda_2018.csv', index=False)